#Your Guide to Managing the Moral Sense Test

*Please note that, although we can obviously link to surveys through Qualtrics and other 3rd-party software, it is STRONGLY PREFERRED that you use code that can interface with SQL-- for instance, Javascript or Flash.*

## How to add a Qualtrics study....

There are **four** places in the html that you need to add you link to. All of these can be found in our webfaction server space. To find these pages, first SFTP in to the server (see Morgan for log in credentials). Then, navigate to webapps > phpmst. This is where the live webpages live.

1. index.html
2. user.php
3. studies.html
4. templates/studies_user.php

Studies can always be found in the **SLIDE 2** section of the page. They are formatted to be displayed in rows of five. Index.html and user.php should only display the top five "most popular" studies, while studies.html and studies_user.php are meant to be full lists of all the studies we have on the website at that time! So, if you're adding a study to index and user, you should simply replace one of the current studies. However, when adding to the studies pages, you can simply add yours as the last in line. You may need to make a new row so that the formatting stays pretty.

The code will look something like this:

In [ ]:
<div class="col-12 col-sm-2 with-hover-text">
	<p><a target="_blank" href="harvard.qualtrics.edu/MYSTUDYLINK"><i class="icon icon-map-marker"></i></p>
	<h2 class="font-semibold">STUDY NAME</h2>
	<h4 class="font-thin">Short enticing blurb here</h4>
	<h4 class="font-semibold">estimated time: 10 minutes</h4></a>
</div><!-- /col12 -->

Note that there are lots of icons to choose from-- try to pick one that provides some visual variation on the page! The full list can be found at the end of this tutorial.

That's all! Adding a study with Qualtrics is pretty painless, but it also doesn't have the advantages of full integration with our system. You won't get demographics or cross-correlations with other measures from users.

##How to add a Javascript or Flash study...

###Frontend


There are **four** places in the html that you need to add you link to. All of these can be found in our webfaction server space. To find these pages, first SFTP in to the server (see Morgan for log in credentials). Then, navigate to webapps > phpmst. This is where the live webpages live.

1. index.html
2. user.php
3. studies.html
4. templates/studies_user.php

Studies can always be found in the **SLIDE 2** section of the page. They are formatted to be displayed in rows of five. Index.html and user.php should only display the top five "most popular" studies, while studies.html and studies_user.php are meant to be full lists of all the studies we have on the website at that time! So, if you're adding a study to index and user, you should simply replace one of the current studies. However, when adding to the studies pages, you can simply add yours as the last in line. You may need to make a new row so that the formatting stays pretty.

The code will look something like this:

In [ ]:
<div class="col-12 col-sm-2 with-hover-text">
	<p><a target="_blank" href="turkgate.php?studyname"><i class="icon icon-map-marker"></i></p>
	<h2 class="font-semibold">STUDY NAME</h2>
	<h4 class="font-thin">Short enticing blurb here</h4>
	<h4 class="font-semibold">estimated time: 10 minutes</h4></a>
</div><!-- /col12 -->

Note that there are lots of icons to choose from-- try to pick one that provides some visual variation on the page! The full list can be found at the end of this tutorial.

We use turkgate.php to check whether or not users (or visitors, within the same session) have already taken this or a similar test. If not, it does one of two things:
1. IF ITS NOT A REGISTERED USER: Directs them to **visitordem.php?studyname**, which collects visitor's demographic information and then redirects them to studyname.html. So, if my experiment lives on empathy.html, my link should be turkgate.php?empathy
2. IF IT IS A USER: Directs them to the study at studyname.html without going through demographics.

**If you need TurkGate functionality amongst a group of studies (rather than just making sure no one is taking the same survey twice), please see Morgan.**

###Setting up the SQL table

Although you're welcome to use your own SQL set up if you're more comfortable doing it that way, again I urge you to follow this tutorial so that the lab can get the most out of this data.

First of all, see Morgan for the log in credentials to the SQL database. There's a fairly good interface that we can use, which can be accessed at [this link](web402.webfaction.com/phpMyAdmin/index.php), if you do not have a preferred SQL client.

For now, while we still have a manageable number of studies and tables, I'd prefer that all MST surveys have their data tables **in the database "mst".** Why? Well, it helps us efficiently insert information in to multiple different tables at once (the way we must in order to keep demographics separate from responses separate from identifying information). **The name of your data table should match the name you used for your experiment web page.** If my survey is at empathy.html, the table name should be empathy.

**Your table should have the following columns:**
1. rt
2. response
3. scenNum (if you're using randomization)
4. trialNum
5. userid
6. visitorid
7. timestamp (time of row insertion)

*Feel free to copy the structure of an existing table in the database. You can do this by clicking on one of the tables (such as empathy) and clicking on More > Operations in the top right corner. The fourth cell of that page will give you the option to copy table structure into a new table with your desired name. Even if you decide not to do this, **please check that your data types are consistent**.*

You'll need to do one more thing in SQL. Go to the table **useremails** and add a column with the name of your study. This column should hold a boolean value that defaults to FALSE or 0. This is the table we use to record which users have taken which surveys. Every time a given user takes a new survey, we look up their email address, and revise the 0 in the test's column to a 1.

###Saving experiment data to SQL

I have added several templates to this repo to illustrate a working experiment. There are three relevant files: **studyname.html**, **parameters_studyname.js** and **saveDataSTUDYNAME.php**. Here's how they work together.

The *parameters* file contains information such as the text of the scenarios you'll be showing (if it's a standard vignette study) and number of trials. The *parameters* file is read in at the top incantation of *studyname.html*. This file is the actual UI for the experiment. My template uses jsPsych to render a simple likert experiment. Consent and debrief, as well as a function that provides feedback to the user (**strongly recommended!**) are included in this template. studyname.html collects data from the experiment, and upon finish sends the data, the name of the SQL table, and variables (such as randomized order) to *saveDataSTUDYNAME.php*. This final script parses the data and collects the session variables (userid and visitorid) in order to generate rows to be inserted into the SQL table specified. Finally, it connects to our database and inserts one row per trial. If the test-taker is a user with an email on file, it also flips the boolean for the current study in the **useremails** database.

_Don't forget to provide summary statistics at the end of your study!_

**Okay, now that I've got my scripts working, where do I put them?**

1. The **studyname.html** file goes in **webapps/phpmst/experiment**. 
2. The **parameters_studyname.js** file goes in **webapps/phpmst/experiment/public/js**
3. The **saveDataSTUDYNAME.php** file goes in **webapps/phpmst/experiment/data**

More information is of course available in the commented code of these templates. Although everyone's javascript interface will be a little bit different, I do think the saveData function will stay essentially the same. 

The most challenging thing about this workflow is accessing the data. jsPsych has a pretty clunky data structure, so you might need to do some debugging and finagling to get your responses to record correctly. See [here](http://docs.jspsych.org/core_library/jspsych-data/) and [here](https://groups.google.com/forum/#!forum/jspsych) for documentation relating to this. 

##Useful SQL Commands and Tidbits

Every once in a while, I find that useremails isn't updating correctly-- that is, not everyone
who takes a test accurately gets a 1 in the corresponding column of useremails. So, periodically, you can run the following SQL command, which should update any rows that were missed in error:

In [ ]:
UPDATE useremails SET disgust = 1 WHERE email in (SELECT email FROM mstusers
                                                      INNER JOIN disgust ON
                                                      disgust.userid = mstusers.userid)

Here, the table containing the data for the survey has the same name as the corresponding column in useremails.

To export data from an entire table, you can simply navigate to that table in phpMyAdmin and click the green 'Export' button near the top of the screen. It's more likely, however, that you'll want to join this data with participants demographics or their answers on other tests. These commands should do the trick.

In [ ]:
SELECT * FROM `impulsivity` INNER JOIN userdem ON userdem.userid = impulsivity.userid
                            WHERE userdem.userid!=0 

In [ ]:
SELECT * FROM `impulsivity` INNER JOIN visitordem ON visitordem.visitorid = impulsivity.visitorid
                            WHERE visitordem.visitorid!=0 

Note that you'll have to run both of these commands to get all of the data! User and visitor demographics are stored in different tables. Also, some of the first rows are from beta version of the site and can be safely dropped if there are lots of zeros.

In [ ]:
SELECT impulsivity.userid, AVG(impulsivity.response),AVG(originaldilemmas.response) 
                            FROM impulsivity INNER JOIN originaldilemmas 
                            ON impulsivity.userid = originaldilemmas.userid 
                            WHERE impulsivity.userid!=0 
                            GROUP BY impulsivity.userid

This gives you a list of averages for the same users on two different tests. You could use these for across-subjects correlations. You would also want to run this command joining on visitorid. AVG is a built-in SQL aggregating function. Of course, you could also get the full list of responses to each test per participant-- just be careful with the join command here! If you just do an inner join, you'll each row of the first test paired with EVERY row of the second test. So you'll get n^2 rows of data per participant, where you only want n.

Once you've run a command like this, scroll to the bottom of the page, where you'll find a cell titled **Query results operations**. Click the export button within this menu to export *just the results from this query*. If you click the export at the TOP of the screen, you'll just get the full table, which probably isn't what you want.

##How to add new demographics to the list

This part is slightly more "adding lots of things on lots of pages" intensive than adding a study is. I am totally happy to take care of this for the time being. However, if you're ambitious and want to do it yourself, follow these steps. Here are the pages you'll need to edit to make this work:

1. visitordem.php
2. submitvisitordem.php
3. register.php
4. submitregister.php
5. update.php
6. user.php
7. js/script.js
8. SQL tables visitordem and userdem

I know, I know. I'm sorry.

Let's start with **visitordem.php**, which contains mostly html and should be the easiest to edit. First, find **SLIDE 5**. This is where the demographics form lives. You should see a bunch of entries of the form `<div class="form-group">`. You're going to make a new div of this class! Here's an example of a drop-down menu:

In [ ]:
<div class="form-group">
    <label for="psych">Please indicate any formal education in Psychology.</label>
	<select class="form-control" name="psych" > #give your entry a unique name!!
		<option> </option> #top is blank for aesthetic and null-defaut reasons
  		<option value="1">Very little or none</option> #the value is what will be passed to SQL
  		<option value="2">Some (e.g. one or two courses)</option> #not the label!
 		<option value="3">A lot (e.g. three or more courses)</option>
  	</select>
  </div>

You can add your new item wherever you want, but **take note of the order of the demographics questions**. It's going to matter later!

Ok, you're done with visitordem.php. Let's edit **submitvisitordem.php** to reflect this change.

Find line 17. It should read `$dem = query("INSERT INTO....")`. This line submits a row of demographics  to the SQL database. The order of what you're submitting must EXACTLY match the order of columns in the SQL table. So for consistency's sake, just have the display order be the same as the column order and thus the same as the insertion order. If I'm submitting a new item called "psych" in between "numkids" and "phil", I'd need to edit the query like so:

In [ ]:
$dem = query("INSERT INTO visitordem (age, gender, education, ses, religion, religious, social, econ, numkids, psych, phil, english, otherlang,country,state,zip) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", $_POST["age"],$_POST["gender"],$_POST["education"],$_POST["ses"],$_POST["religion"],$_POST["religious"],$_POST["social"],$_POST["econ"],$_POST["numkids"],$_POST["psych"],$_POST["phil"],$_POST["english"],$_POST["otherlang"],$_POST["country"],$_POST["state"],$_POST["zip"]);


Don't forget to add another question mark within the parentheses after VALUES.

Next, jump down the page and find line 40. We're defining a bunch of session variables here. This is so that the registration form and the user profile can accurately auto-populate. You'll need to add one for your new variable, like so:

In [ ]:
$_SESSION["psych"] = $_POST["psych"];

Add the same line again on line 74! That's it for submitvisitordem.php. Next up is **register.php**. This is a little more difficult to edit, because we fields to be automatically filled in if the person is deciding to register after already submitting their information on the visitor demographics form! So there are two sections you need to change here. The first is the php section at the top of **Slide 5**, around line 100. Here where we're creating a bunch of local variables from the session variables we defined above. Create a new one for the variable you're adding.

In [ ]:
if(isset($_SESSION["psych"]))
		$psych = $_SESSION["psych"];
	else
		$psych= " ";

Now skip down to the corresponding html-ish section. Remember to use the same order that your programmed in visitordem.php! You can mostly reuse your code from there, but you need to program a loop that auto-selects the option that matches the current session variable. Your code should look something like this:

In [ ]:
<div class="form-group">
    <label for="psych">Please indicate any formal education in Psychology.</label>
	<select class="form-control" name="psych">
		#make the session variable the default selected value
        <?php
    		$index = 0;
    		$strings = array("","Very little or none","Some (e.g. one or two courses)","A lot (e.g. three or more courses)");
    		
			while($index<4){
			
			$currString = $strings[$index];
				if ($index == $psych)
				{
            		echo "<option value =\"".$index."\" selected=\"selected\">".$currString."</option>";
            		
        		}
        		else{
            		echo "<option value =\"".$index."\" >".$currString."</option>";
        		}
        	$index++;
        	}
		?>
  	</select>
  </div>

Let's make the corresponding edits in **submitregister.php** on line 71ish:

In [ ]:
$_SESSION["psych"] = $_POST["psych"];

and on line 96ish:

In [ ]:
$dem = query("INSERT INTO userdem (userid, age, gender, education, ses, religion, religious, social, econ, numkids, psych, phil, english, otherlang,country,state,zip,contact) VALUES (?, ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", $userid, $_POST["age"],$_POST["gender"],$_POST["education"],$_POST["ses"],$_POST["religion"],$_POST["religious"],$_POST["social"],$_POST["econ"],$_POST["numkids"],$_POST["psych"],$_POST["phil"],$_POST["english"],$_POST["otherlang"],$_POST["country"],$_POST["state"],$_POST["zip"],$_POST["contact"]);

Let's move on to **user.php** and **update.php**! The edits you make here will be completely analogous to what you've just done with register.php and submitregister.php. Again, in user.php on line 100ish, add your new local variable:

In [ ]:
if(isset($_SESSION["psych"]))
		$psych = $_SESSION["psych"];
	else
		$psych= " ";

Now the rest of this page is written completely in php. Why? We want to be able to redirect both registered users and visitors within the same session to this page. But only users should have the priviledge of updating their demographic profile and seeing their Shiny app in-line on their profile. So, essentially, you're just going to echo all of the html you need. This will result in some annoying quotation escaping issues. See below for sample code. Again, **keep order consistent**.

In [ ]:
echo "<div class=\"form-group\">
    <label for=\"psych\">Please indicate any formal education in Psychology.</label>
	<select class=\"form-control\" name=\"psych\">";
		$index = 0;
    	$strings = array("","Very little or none","Some (e.g. one or two courses)","A lot (e.g. three or more courses)");
    		
		while($index<4){
			
		$currString = $strings[$index];
		if ($index == $psych)
		{
            echo "<option value =\"".$index."\" selected=\"selected\">".$currString."</option>";
            		
        }
        else{
            echo "<option value =\"".$index."\" >".$currString."</option>";
        }
        $index++;
        }
        	
			
echo "</select></div>"

You're echoing everything up until you set the variable $index=0. That part is the beginning of the php. To reiterate, you're using the php echo function to output all of the pure html, and then when you need to do a loop, you use php.

Now, in **update.php**, just add a new if statement for your variable:

In [ ]:
if (!empty($_POST["psych"]))
        {
            $dem = query("UPDATE userdem SET psych = ? WHERE userid = ?", $_POST["psych"], $_SESSION["userid"]);
            $_SESSION["psych"] = $_POST["psych"];
            if ($dem === "false")
			{	
				apologize("An error occured.");
			}   
        }

Okay, you're getting close now. Only a few more things left to add. Open up **script.js** which lives in the *js* folder. You'll need to edit this file in a few places, but luckily you're adding the same line in each of those places. That line is something like:

In [ ]:
psych: $('[name="psych"]').val(),

Where does it go? We need it in four sections:
1. Visitor alert to register function(isConfirm) if statement
2. Visitor alert to register function(isConfirm) else statement
2. Registration confirmation
3. Update Success

Look for a bunch of lines of code that correspond to all of our demographic variables. They should be wrapped within a `$.ajax` post request. Simply add your new variable to the list in each of these places. Order doesn't particularly matter, but **don't forget the comma**!! If you demographic item is something like a radio button, you'll have to choose a more complicated selector like

In [ ]:
religious: $('input:radio[name=religious]:checked').val(),

Here's the final part! Use phpMyAdmin or your SQL client to get in to the SQL database for the MST. Add a column with the name of your value to **userdem** *and* **visitordem**. ORDER IS IMPORTANT, so please look back at the query functions in submitvisitordem.php and submitregister.php if you've forgotten where you specified this item to be. Make sure that your column is inserted correctly-- in the case of this "psych" variable, I want to put it in between "numkids" and "phil" in the sql tables. Check the structure of other variables in the table (by clicking the *Structure* button near the top of the SQL interface) if you're unsure how to code for integer or character variables and whether or not to specify a default VALUE.

##How to email users

Stay tuned, this is coming soon! We may end up using a third-party email client so that our emails don't get tagged as spam.

##What's up with the Shiny thing?

Currently, our app is only configured to visualize data from the first three tests that I coded: trolley problems, disgust scale, and empathy scale. Those three are all set up to interface nicely with it, so you shouldn't have to mess with it at all.

Of course, in the future it would be really great if we could visualize everything, or at least change it up a bit! There are a few problems with the current Shiny set up, which I will enumerate here so that someone smarter than me can figure out how to fix them further down the line. :)

Currently, our shiny app is hosted at moral.shinyapps.io. We pay for 500 hours of usage a month, and the hosting is handled by that third party. Currently, the account is under Morgan's email address, but hopefully she'll remember that fix that soon... Anyway, because we shunt users over to this other website in order to view their results, there's no way for the app to connect with our SQL database, which is hosted by webfaction (the same server where the website lives). Essentially, the way we've dealt with this is to scrape the SQL database for the user's average scores on each of the three tests in submitDataSTUDYNAME.php. We then pass all of the scores (if they exist) as variables in the query string to Shiny. Once the *current* data has been successfully inserted in SQL, we *return* a link (such as **moral.shinyapps.io?curr=empathy&empav=3.5&disav=2.3&morav=4.4**) which the window redirects to. 

The app is configured the parse the query string and plot based on those values. But this solution is not ideal for several reasons. First of all, we're already paying for server space! We shouldn't need to pay for extra app hosting. Second of all, because we're limited in what data we can pass through the query string, there's no way to generate distributions of *current MST data* upon which to juxtapose the user's averages. Right now the graphs are generated from historical data from Fiery's original Flash experiments.

Now, there *is* a way to **host R Shiny on your own server**, but after a few failed attempts over the summer I gave up on that project. Ultimately, though, this is the way we need to go. Once the hosting is straightened out, we'll have a lot more flexibility about what we can plot!

**tl;dr We can't do a lot of customization with the Shiny app at the moment. Until we can host it on the webfaction server, we should leave it alone. But everyone SHOULD provide summary statistics at the end of their surveys!!! (A function that does this is included in the template code.)**

##Full list of available icons

.icon-glass
.icon-music
.icon-search
.icon-envelope-alt
.icon-heart
.icon-star
.icon-star-empty
.icon-user
.icon-film
.icon-th-large
.icon-th
.icon-th-list
.icon-ok
.icon-remove
.icon-zoom-in
.icon-zoom-out
.icon-power-off
.icon-signal
.icon-gear
.icon-trash
.icon-home
.icon-file-alt
.icon-time
.icon-road
.icon-download-alt
.icon-download
.icon-upload
.icon-inbox
.icon-play-circle
.icon-rotate-right
.icon-refresh
.icon-list-alt
.icon-lock
.icon-flag
.icon-headphones
.icon-volume-off
.icon-volume-down
.icon-volume-up
.icon-qrcode
.icon-barcode
.icon-tag
.icon-tags
.icon-book
.icon-bookmark
.icon-print
.icon-camera
.icon-font
.icon-bold
.icon-italic
.icon-text-height
.icon-text-width
.icon-align-left
.icon-align-center
.icon-align-right
.icon-align-justify
.icon-list
.icon-indent-left
.icon-indent-right
.icon-facetime-video
.icon-picture
.icon-pencil
.icon-map-marker
.icon-adjust
.icon-tint
.icon-edit
.icon-share
.icon-check
.icon-move
.icon-step-backward
.icon-fast-backward
.icon-backward
.icon-play
.icon-pause
.icon-stop
.icon-forward
.icon-fast-forward
.icon-step-forward
.icon-eject
.icon-chevron-left
.icon-chevron-right
.icon-plus-sign
.icon-minus-sign
.icon-remove-sign
.icon-ok-sign
.icon-question-sign
.icon-info-sign
.icon-screenshot
.icon-remove-circle
.icon-ok-circle
.icon-ban-circle
.icon-arrow-left
.icon-arrow-right
.icon-arrow-up
.icon-arrow-down
.icon-mail-forward
.icon-resize-full
.icon-resize-small
.icon-plus
.icon-minus
.icon-asterisk
.icon-exclamation-sign
.icon-gift
.icon-leaf
.icon-fire
.icon-eye-open
.icon-eye-close
.icon-warning-sign
.icon-plane
.icon-calendar
.icon-random
.icon-comment
.icon-magnet
.icon-chevron-up
.icon-chevron-down
.icon-retweet
.icon-shopping-cart
.icon-folder-close
.icon-folder-open
.icon-resize-vertical
.icon-resize-horizontal
.icon-bar-chart
.icon-twitter-sign
.icon-facebook-sign
.icon-camera-retro
.icon-key
.icon-gears
.icon-comments
.icon-thumbs-up-alt
.icon-thumbs-down-alt
.icon-star-half
.icon-heart-empty
.icon-signout
.icon-linkedin-sign
.icon-pushpin
.icon-external-link
.icon-signin
.icon-trophy
.icon-github-sign
.icon-upload-alt
.icon-lemon
.icon-phone
.icon-unchecked
.icon-bookmark-empty
.icon-phone-sign
.icon-twitter
.icon-facebook
.icon-github
.icon-unlock
.icon-credit-card
.icon-rss
.icon-hdd
.icon-bullhorn
.icon-bell
.icon-certificate
.icon-hand-right
.icon-hand-left
.icon-hand-up
.icon-hand-down
.icon-circle-arrow-left
.icon-circle-arrow-right
.icon-circle-arrow-up
.icon-circle-arrow-down
.icon-globe
.icon-wrench
.icon-tasks
.icon-filter
.icon-briefcase
.icon-fullscreen
.icon-group
.icon-link
.icon-cloud
.icon-beaker
.icon-cut
.icon-copy
.icon-paperclip
.icon-save
.icon-sign-blank
.icon-reorder
.icon-list-ul
.icon-list-ol
.icon-strikethrough
.icon-underline
.icon-table
.icon-magic
.icon-truck
.icon-pinterest
.icon-pinterest-sign
.icon-google-plus-sign
.icon-google-plus
.icon-money
.icon-caret-down
.icon-caret-up
.icon-caret-left
.icon-caret-right
.icon-columns
.icon-sort
.icon-sort-down
.icon-sort-up
.icon-envelope
.icon-linkedin
.icon-rotate-left
.icon-legal
.icon-dashboard
.icon-comment-alt
.icon-comments-alt
.icon-bolt
.icon-sitemap
.icon-umbrella
.icon-paste
.icon-lightbulb
.icon-exchange
.icon-cloud-download
.icon-cloud-upload
.icon-user-md
.icon-stethoscope
.icon-suitcase
.icon-bell-alt
.icon-coffee
.icon-food
.icon-file-text-alt
.icon-building
.icon-hospital
.icon-ambulance
.icon-medkit
.icon-fighter-jet
.icon-beer
.icon-h-sign
.icon-plus-sign-alt
.icon-double-angle-left
.icon-double-angle-right
.icon-double-angle-up
.icon-double-angle-down
.icon-angle-left
.icon-angle-right
.icon-angle-up
.icon-angle-down
.icon-desktop
.icon-laptop
.icon-tablet
.icon-mobile-phone
.icon-circle-blank
.icon-quote-left
.icon-quote-right
.icon-spinner
.icon-circle
.icon-mail-reply
.icon-github-alt
.icon-folder-close-alt
.icon-folder-open-alt
.icon-expand-alt
.icon-collapse-alt
.icon-smile
.icon-frown
.icon-meh
.icon-gamepad
.icon-keyboard
.icon-flag-alt
.icon-flag-checkered
.icon-terminal
.icon-code
.icon-reply-all
.icon-mail-reply-all
.icon-star-half-full
.icon-location-arrow
.icon-crop
.icon-code-fork
.icon-unlink
.icon-question
.icon-info
.icon-exclamation
.icon-superscript
.icon-subscript
.icon-eraser
.icon-puzzle-piece
.icon-microphone
.icon-microphone-off
.icon-shield
.icon-calendar-empty
.icon-fire-extinguisher
.icon-rocket
.icon-maxcdn
.icon-chevron-sign-left
.icon-chevron-sign-right
.icon-chevron-sign-up
.icon-chevron-sign-down
.icon-html5
.icon-css3
.icon-anchor
.icon-unlock-alt
.icon-bullseye
.icon-ellipsis-horizontal
.icon-ellipsis-vertical
.icon-rss-sign
.icon-play-sign
.icon-ticket
.icon-minus-sign-alt
.icon-check-minus
.icon-level-up
.icon-level-down
.icon-check-sign
.icon-edit-sign
.icon-external-link-sign
.icon-share-sign
.icon-compass
.icon-collapse
.icon-collapse-top
.icon-expand
.icon-euro
.icon-gbp
.icon-dollar
.icon-rupee
.icon-yen
.icon-renminbi
.icon-won
.icon-bitcoin
.icon-file
.icon-file-text
.icon-sort-by-alphabet
.icon-sort-by-alphabet-alt
.icon-sort-by-attributes
.icon-sort-by-attributes-alt
.icon-sort-by-order
.icon-sort-by-order-alt
.icon-thumbs-up
.icon-thumbs-down
.icon-youtube-sign
.icon-youtube
.icon-xing
.icon-xing-sign
.icon-youtube-play
.icon-dropbox
.icon-stackexchange
.icon-instagram
.icon-flickr
.icon-adn
.icon-bitbucket
.icon-bitbucket-sign
.icon-tumblr
.icon-tumblr-sign
.icon-long-arrow-down
.icon-long-arrow-up
.icon-long-arrow-left
.icon-long-arrow-right
.icon-apple
.icon-windows
.icon-android
.icon-linux
.icon-dribbble
.icon-skype
.icon-foursquare
.icon-trello
.icon-female
.icon-male
.icon-gittip
.icon-sun
.icon-moon
.icon-archive
.icon-bug
.icon-vk
.icon-weibo
.icon-renren